In [10]:
import timm

def print_model_layers(model, prefix=''):
    for name, module in model.named_children():
        if len(list(module.children())) > 0:
            print_model_layers(module, prefix + name + '.')
        else:
            print(f'{prefix +"sep---------------"+ name}: {module}')

# Load the model
model = timm.create_model("hf_hub:timm/maxvit_tiny_tf_224.in1k", pretrained=False)

# Print all layers and sub-modules of the model
print("Printing all layers and sub-modules of the model:")
print_model_layers(model)


Printing all layers and sub-modules of the model:
stem.sep---------------conv1: Conv2dSame(3, 64, kernel_size=(3, 3), stride=(2, 2))
stem.norm1.sep---------------drop: Identity()
stem.norm1.sep---------------act: GELUTanh()
stem.sep---------------conv2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
stages.0.blocks.0.conv.shortcut.sep---------------pool: AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
stages.0.blocks.0.conv.shortcut.sep---------------expand: Identity()
stages.0.blocks.0.conv.pre_norm.sep---------------drop: Identity()
stages.0.blocks.0.conv.pre_norm.sep---------------act: Identity()
stages.0.blocks.0.conv.sep---------------down: Identity()
stages.0.blocks.0.conv.sep---------------conv1_1x1: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
stages.0.blocks.0.conv.norm1.sep---------------drop: Identity()
stages.0.blocks.0.conv.norm1.sep---------------act: GELUTanh()
stages.0.blocks.0.conv.sep---------------conv2_kxk: Conv2

In [14]:
import timm
import torch
from torch import nn

def print_model_layers(model, prefix=''):
    """
    Prints all layers and sub-modules of a PyTorch model.

    Args:
    - model (torch.nn.Module): The model to inspect.
    - prefix (str): A prefix for layer naming, used for nested models.
    """
    for name, module in model.named_children():
        if len(list(module.children())) > 0:  # If the module has children, recursively print its layers
            print_model_layers(module, prefix + name + '.')
        else:
            print(f'{prefix + name}: {module}')

def get_submodel(model, layer_name):
    """
    Extracts a submodel from the original model up to a specified layer.

    Args:
    - model (torch.nn.Module): The original PyTorch model.
    - layer_name (str): The name of the layer up to which the submodel should be extracted.

    Returns:
    - torch.nn.Module: A submodel that ends at the specified layer.
    """
    submodel = nn.Sequential()
    for name, module in model.named_children():
        submodel.add_module(name, module)
        if name == layer_name:
            break
        # If the current module has children, recurse into it to find the layer
        elif len(list(module.children())) > 0:
            submodel[-1] = get_submodel(module, layer_name)
            if submodel[-1] is not None:
                break
    return submodel

# Load the model
model_name = "hf_hub:timm/maxvit_tiny_tf_224.in1k"
model = timm.create_model(model_name, pretrained=False)

# Print all layers and sub-modules of the model
print("Printing all layers and sub-modules of the model:")
print_model_layers(model)

# Input for layer name
layer_name = input("Enter the layer name up to which you want the submodel: ")

# Extract and print the submodel
submodel = get_submodel(model, layer_name)
if submodel is not None:
    print(f"\nSubmodel up to layer '{layer_name}':")
    print(submodel)
else:
    print(f"Layer '{layer_name}' not found in the model.")


Printing all layers and sub-modules of the model:
stem.conv1: Conv2dSame(3, 64, kernel_size=(3, 3), stride=(2, 2))
stem.norm1.drop: Identity()
stem.norm1.act: GELUTanh()
stem.conv2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
stages.0.blocks.0.conv.shortcut.pool: AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
stages.0.blocks.0.conv.shortcut.expand: Identity()
stages.0.blocks.0.conv.pre_norm.drop: Identity()
stages.0.blocks.0.conv.pre_norm.act: Identity()
stages.0.blocks.0.conv.down: Identity()
stages.0.blocks.0.conv.conv1_1x1: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
stages.0.blocks.0.conv.norm1.drop: Identity()
stages.0.blocks.0.conv.norm1.act: GELUTanh()
stages.0.blocks.0.conv.conv2_kxk: Conv2dSame(256, 256, kernel_size=(3, 3), stride=(2, 2), groups=256, bias=False)
stages.0.blocks.0.conv.norm2.drop: Identity()
stages.0.blocks.0.conv.norm2.act: GELUTanh()
stages.0.blocks.0.conv.se.fc1: Conv2d(256, 16, kernel_size=(1, 1), s

In [16]:
# Input for layer name
layer_name = input("Enter the layer name up to which you want the submodel: ")

# Extract and print the submodel
submodel = get_submodel(model, layer_name)
if submodel is not None:
    print(f"\nSubmodel up to layer '{layer_name}':")
    print(submodel)
else:
    print(f"Layer '{layer_name}' not found in the model.")

from torchsummary import summary

# Assuming the model variable contains your model
summary(submodel, input_size=(3, 224, 224))

Enter the layer name up to which you want the submodel: stages.2.blocks.1.attn_block.mlp.fc1

Submodel up to layer 'stages.2.blocks.1.attn_block.mlp.fc1':
Sequential(
  (stem): Sequential(
    (conv1): Conv2dSame(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (norm1): Sequential(
      (drop): Identity()
      (act): GELUTanh()
    )
  )
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
        Conv2dSame-1         [-1, 64, 112, 112]           1,792
          Identity-2         [-1, 64, 112, 112]               0
          GELUTanh-3         [-1, 64, 112, 112]               0
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 18.38
Params size (MB): 0.01
Estimated Total Size (MB): 18.96
----------------------------------------------------------------
